In [1]:
# downgrade pytorch for incompatibility problem
!pip install torch==1.7.0

     |████████████████████████████████| 776.7 MB 4.3 kB/s 
  Attempting uninstall: torch
    Found existing installation: torch 1.9.0+cu111
    Uninstalling torch-1.9.0+cu111:
      Successfully uninstalled torch-1.9.0+cu111
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.10.0+cu111 requires torch==1.9.0, but you have torch 1.7.0 which is incompatible.
torchtext 0.10.0 requires torch==1.9.0, but you have torch 1.7.0 which is incompatible.


In [2]:
!pip install transformers
!pip install seqeval
!pip install sentencepiece

import os
import json
import logging

import numpy as np
import torch
from torch.nn import functional as F
import torch.nn as nn 

import sentencepiece as spm
from seqeval.metrics import precision_score as seq_precision, recall_score as seq_recall, f1_score as seq_f1
from transformers import AutoTokenizer, XLMRobertaModel, XLMRobertaForMaskedLM
from transformers import AdamW, get_linear_schedule_with_warmup


from tqdm.notebook import tqdm
from easydict import EasyDict
import gc
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
import pickle

logger = logging.getLogger(__name__)

     |████████████████████████████████| 3.1 MB 12.6 MB/s 
     |████████████████████████████████| 596 kB 53.1 MB/s 
     |████████████████████████████████| 56 kB 4.9 MB/s 
     |████████████████████████████████| 895 kB 54.3 MB/s 
     |████████████████████████████████| 3.3 MB 37.5 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 43 kB 1.6 MB/s 
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16181 sha256=2ae0458096ffaed9d79823f2a5a75aaf78e35d8b412cdd58656e8abdc5f4d3a8
  Stored in directory: /root/.cache/pip/wheels/05/96/ee/7cac4e74f3b19e3158dce26a20a1c86b3533c43ec72a549fd7
Successfully built seqeval
     |████████████████████████████████| 1.2 MB 12.2 MB/s 


In [3]:
import zipfile
with zipfile.ZipFile('/content/drive-download-20211030T133316Z-001.zip') as zf:
    zf.extractall()

# Model

In [4]:
def num_parameters(parameters):
    num = 0
    for i in parameters:
        num += len(i)
    return num

class Detector(nn.Module):
    def __init__(self, input_dim, output_dim,  embedding_dim, num_layers, hidden_size):

        super(Detector, self).__init__()
        self.input_dim = input_dim
        self.output_dim = output_dim
        self.embedding_dim  = embedding_dim
        self.num_layers = num_layers
        self.hidden_size = hidden_size
        self.embedding = nn.Embedding(num_embeddings = self.input_dim, embedding_dim = self.embedding_dim, )
        self.LSTM = nn.LSTM(input_size = self.embedding_dim, hidden_size= self.hidden_size, num_layers = self.num_layers, 
                            batch_first = True, dropout = 0.1, bidirectional = True)
        self.linear = nn.Linear(self.hidden_size*2, self.output_dim)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        emb = self.embedding(x)
        outputs, (h_n, h_c) = self.LSTM(emb)
        logits = self.linear(outputs)

        p = self.sigmoid(logits)
        return p


class HardMasked(nn.Module):
    def __init__(self, detector, MaskedLM, detector_tokenizer, maskedlm_tokenzier,device ):
        super(HardMasked, self).__init__()

        self.detector = detector.to(device)
        self.MaskedLM = MaskedLM.to(device)
        self.detector_tokenizer = detector_tokenizer
        self.maskedlm_tokenizer = maskedlm_tokenizer
        self.use_device = device


    def forward(self, s):
        maskedlm_features = self.prepare_input(s)
        outputs = MaskedLM(input_ids = torch.tensor([maskedlm_features['input_ids']], dtype = torch.long, device = self.use_device), 
                            attention_mask = torch.tensor([maskedlm_features['attention_mask']], dtype = torch.long, device = self.use_device) )
        logits = outputs['logits'][0]
        output_ids = torch.argmax(logits, dim = -1)
        final_output = maskedlm_tokenizer.decode(output_ids)
        return final_output
        
    def prepare_input(self, s):

        detector_input_ids = self.detector_tokenizer.encode(s, out_type = int)
        detector_input_pieces = self.detector_tokenizer.id_to_piece(detector_input_ids)
        detector_outputs = (self.detector(torch.tensor([detector_input_ids], dtype = torch.long, device = self.use_device))[0].reshape(1,-1) > 0.5).int()[0] 

        for i in range(1, len(detector_input_pieces)):
            if detector_outputs[i] == 1:
                detector_input_pieces[i] = ' <mask>'

        masked_s = self.detector_tokenizer.decode(detector_input_pieces)
        for i in range(5):
            masked_s = re.sub(r'<mask>\s<mask>', '<mask>', masked_s)

        maskedlm_features = maskedlm_tokenizer(masked_s)

        return maskedlm_features

        
        

In [5]:
# Load detector and XLM-R masked language model to create Hard-Masked XLM-R
# Change the directories to Detector967.pkl and spm_tokenizer.model 

detector_path = '/content/Detector967.pkl'
detector_tokenizer_path = '/content/spm_tokenizer.model'

MaskedLM = XLMRobertaForMaskedLM.from_pretrained('xlm-roberta-base')

maskedlm_tokenizer = AutoTokenizer.from_pretrained('xlm-roberta-base')

detector_tokenizer = spm.SentencePieceProcessor(detector_tokenizer_path, )

detector = torch.load(detector_path)

Downloading:   0%|          | 0.00/512 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.04G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.83M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/8.68M [00:00<?, ?B/s]

In [6]:
model = HardMasked(detector, MaskedLM, detector_tokenizer, maskedlm_tokenizer, 'cuda')

In [7]:
s = 'Tôi vẫn luôn iu cô ấy với hết tấm lòng của mk'
model(s)

In [16]:
# không biết lách cách là gì nên model đã replace nó bằng xa cách!
# nó bỏ cả chữ "còn" ra để thay bằng dấu phẩy ~_~
s = 'xe đạp lách cách tôi vẫn chưa quen, đường thì tối chơi vơi còn tôi vẫn cứ đứng đợi'
model(s)

'<s> xe đạp xa cách tôi vẫn chưa quen, đường thì tối chơi vơi, tôi vẫn cứ đứng đợi</s>'

In [9]:
s = 'Đi theo anh đến mội chân trời'
model(s)

'<s> Đi theo anh đến tận chân trời</s>'